# 第8章 字典和集合
## 8.1 数据存储,检索和字典
计算机的基本功能是存储和处理数据,首先是存储数据,处理也需要先找到被处理的数据,或成为**访问数据**.所以,数据的存储和访问是计算机最基本的功能.
### 8.1.1 数据存储和检索
数据访问的基本方式是基于存储位置.

找到数据的存储位置的操作称为__检索__,类似查字典.
#### 概述
数据检索牵涉到两个方面,一方面是已存储的数据集合,另一方面是用户检索时提供的信息.具体检索可以是确定特定数据是否存在于数据集中,相当于集合成员判断;也可以是希望找到与所提供信息相关的数据,类似与在字典里查询词语的解释.在后一方式中,检索时提供的信息被看做`检索码`或`关键码`(key).这种关键码也常作为数据的一部分,存储在数据集里,这就是**基于关键码的数据存储和检索**,是本章讨论的主题.

作为检索基础的关键码,通常是数据项的某种(可能具有唯一性的)特征,可以是数据内容的一个组成部分,也可以是专门为数据检索建立的标签.后一种情况实际中也很常见.以学校的学生记录为例,原本只需要记录与学生相关的信息.而为每个学生指定一个学号,只是为了检索方便.这一类关键码通常是数据的唯一`标识`.

基本假设(抽象的):需要存储的数据元素由两个部分组成,其中一部分是与检索相关的关键码,另一部分是与之关联的数据.

`字典`就是支持基于关键码的数据存储与检索的数据结构.也被称为`查找表`,`映射`或者`关联表`等.字典实现数据的存储和检索,而需要存储和检索的信息和环境有许多具体情况,因此可能要求不同的实现技术.

在字典的使用中,最重要也是使用最频繁的操作就是检索(search,也称查找).
#### 字典操作和效率
实际中使用的字典可以分为两类:
- 静态字典:在建立之后,这种字典内容和结构都不再变化,主要操作只有检索.对这种字典,需要考虑创建的代价,但最重要的是检索效率.
- 动态字典:在初始创建之后,这种字典的内容(和结构)将一直处于动态变动之中.对这种字典,除了检索之外,最重要的基本操作还包括数据项的插入和删除等.

对于动态字典,还有一个问题也必须重视:这里的插入和删除操作可能导致字典结构的变化.良好设计的字典应该保证其性能不会随着操作的进行而逐渐恶化.

在字典中检索,最终将得到一个结果,或是检索成功而且得到了所需的数据;或是确认了要找的数据不存在,此时可能返回某种特殊标志.有关检索效率的评价标准,通常考虑的是在一次完整检索过程中比较关键码的平均次数,通常称为`平均检索长度`(Average Search Length,ASL),其定义是(其中n为字典中的数据项数):$$ASL=\sum_{i=0}^{n-1}{p_i\cdot{c_i}}$$其中$c_i$和$p_i$分别为第i项数据元素的检索长度和检索概率.上述定义中只考虑了被检索关键码在字典中存在的情况(正确的检索算法保证存在的关键码一定能找到),在很多情况中,还需要考虑字典中不存在被检索关键码的情况.\

#### 字典和索引
实际上,字典是两种功能的统一:
- 作为一种数据存储结构,支持在字典里存储一批数据项.
- 提供支持数据检索的功能,设法维护从关键码找到相关数据的联系信息.

后一功能称为`索引`,其存在的目的就是为检索服务.

做基于关键码的检索,就是要实现从关键码到数据存储位置的映射,而这种映射也就是索引.但是,索引结构本身并不存储数据,只提供(基于关键码的)检索功能,因此它只作为字典的附属结构,不可能独立存在.
### 8.1.2 字典的实现问题
#### 字典抽象数据类型
|ADT Dict:|# 字典抽象数据类型|
|---|---|
|Dict(self)|# 字典构造函数,创建一个新字典|
|is_empty(self)|# 判断self是否为一个空字典|
|num(self)|# 获知字典中的元素个数|
|search(self,key)|# 检索字典里key的关联数据|
|insert(self,key,value)|# 将关联(key,value)加入字典|
|delete(self,key)|# 删除字典中关键码为key的元素|
|values(self)|# 支持以迭代方式取得字典里保存的各项关联中的value|
|entries(self)|# 支持以迭代方式获得字典的关联中的逐堆key和value二元组|

注意,各种字典都不应该允许修改字典关联中的关键码,因为关键码用于确定其所在项在字典里存储的位置,以支持高效检索.如果允许修改关键码,就可能破坏字典数据结构的完整性,导致后续检索操作失败.
#### 字典元素:关联
在支持基于关键码的存储与检索的字典里,一个数据项可以划分为两个部分:其一是与检索有关的关键码部分,另外就是与检索无关的其他数据部分.下面将始终把字典里的数据项简单地分为两部分,一部分是与检索有关的关键码,另一部分称为值,与检索和其他操作(例如插入/删除)的实现方式无关,但在实际应用中却可能非常重要.这样,一个数据项就是一种`二元组`,下面称之为`关联`.


In [1]:
class Assoc:
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def __lt__(self, other):
        return self.key < other.key
    def __le__(self, other):
        return self.key < other.key or self.key == other.key
    def __str__(self):      # 定义字符串表示形式便于输出和交互
        return "Assoc({0},{1})".format(self.key, self.value)

#### 字典的实现
字典实现技术:线性表,连续表,散列表
## 8.2 字典线性表实现
线性表里可以存储信息,因此可以作为字典的实现基础
### 8.2.1 基本实现
基于线性表的字典实现,优点和缺点都非常明显:
- 数据结构和算法都很简单,检索,删除等操作中只需要比较关键码相同(或不同),适用于任何关键码类型(例如,并不要求关键码集合存在某种顺序关系)
- 平均检索效率第(线性时间),表长度n较大时,检索很耗时.
- 删除操作的效率也比较地,因此不太适合频繁变动的字典.

另外,在字典的动态变化中,这种字典的各种操作的效率不变.但这并不时什么优点,因为它们都已经时效率最低的操作了.
### 8.2.2 有序线性表和二分法检索
具有合理内部结构的字典上有可能实现高效检索.

如果关键码取自一个`有序集合`(存在某种内在的序,例如整数的小于等于关系,字符串的字典序等),就可以按照关键码大小的顺序排列字典里的项(从小到大或从大到小).在数据项按序排列时,可以采用`二分法`实现快速检索.

In [2]:
def bisearch(lst, key):
    low, high = 0, len(lst)-1
    while low <= high:        # 范围内还要元素
        mid = low + (high - low) // 2
        if key == lst[mid].key:
            return lst[mid].value
        if key < lst[mid].key:
            high = mid - 1    # 在低半区继续
        else:
            low = mid + 1     # 在高半区继续

#### 二分法检索实例
用二叉树表示对表中所有元素的检索过程,这样的树被称为`二分法检索过程的判定树`.树中结点所标的数时数据项的关键码.

采用有序的顺序表和二分检索法:
- 主要优点时检索速度块,$O(log n)$.
- 插入和删除时需要维护数据项的顺序,因此都是O(n)操作(虽然检索插入/删除位置可以用二分法,为O(log n)时间,但完成操作需要移动元素,为O(n)时间
- 二分技术只能用于关键码可以排序,数据项按关键码排序的字典,而且只适用于顺序存储结构,需要连续的存储块,不适合实现很大的动态字典
### 8.2.3 字典线性表总结
虽然也可以考虑基于单链表或双链表技术实现字典,但通过分析可知:
- 如果字典里的数据项任意排列,插入时可以简单地在表头插入,是O(1)操作;检索和删除都要扫描整个表,是O(n)操作.
- 如果表中的数据项按关键码升序或者降序排列,插入需要检索正确位置,为O(n)操作;检索和删除同样需要顺序扫描检查,平均检查半个表,为O(n)操作.

由上述分析可知:采用链接表实现字典没有任何明显的优势,而且无法利用关键码排序的价值.
#### 问题和思考
采用线性表技术实现字典,只能适合一部分简单需求,如字典的规模比较小,而且不常出现动态操作的情况.但在实际中,各种应用经常需要存储和检索很大的数据集合,字典中的数据内容不断动态变化的情况也很常见.采用简单的顺序表或链接表实现字典,顺序检索的效率太低,在效率上常常不能满足实际应用的需要.采用排序的顺序表和二分检索能大大提高检索速度,但仍有两大问题:
- 不能很好地支持数据地变化(数据插入和删除地效率低)
- 必须采用连续方式存储整个数据集合.如果数据集很大,连续存储方式就很难接受了.

如果要支持存储很大地,经常变动的数据集合,而且希望高效检索,必须考虑其他组织方式.人们为此开发了另外一些结构,主要分为两类:
- 基于散列(hash)思想的`散列表(也称为`哈希表`)
- 基于各种树形结构的数据存储和检索技术(利用树结构的特性,即在不大的深度范围内可以容纳巨大数量的结点)
## 8.3 散列和散列表
在计算机科学技术领域使用非常广泛的散列技术及其在字典方面的应用,即所谓的`散列表`(hash table).
### 8.3.1 散列的思想和应用
在什么情况下基于关键码能最快找到所需的数据?

如果数据项连续存储,而关键码就是存储数据的地址(或下标)!显然,在这种情况下,只需O(1)时间就能得到所需要的数据了.

但是,一般而言,字典所用的关键码可能不是整数,由此不能作为下标.另一方面,即使关键码是整数,也可能因为取值范围太大而不适合作为下标.

散列表的基本思想:如果一种关键码不能或者不适合作为数据存储的下标,可以考虑通过一个变换(一个计算)把它们映射到一种下标.这样做,就把基于关键码的检索转变为基于整数下标的直接元素访问,有可能得到一种高效的字典实现技术.

以散列表的思想实现字典,具体方法是:
 1. 选定一个整数的下标范围(通常以0或1开始),建立一个包括相应元素位置范围的顺序表.
 2. 选定一个从实际关键码集合到上述下标范围的适当映射h:
     - 在需要存入关键码为key的数据时,将其存入表中第h(key)个位置.
     - 遇到以key为关键码检索数据时,直接去找表中第h(key)个位置的元素.
     
这个h称为`散列函数`,也常被称为哈希(hash)函数或杂凑函数,它就是从可能的关键码集合到一个整数区间(下标区间)的映射
#### 散列思想在信息领域的应用
所谓`散列`,一般而言,就是以某种精心设计的方式,从一段可能很长的数据生成一段很短(经常为固定长度)的信息串,例如简单的整数或字符串,最终都是二进制串,而后利用这种二进制串做某种有用的事情.散列技术在计算机和信息技术领域非常有价值,应用极广,可能用在各种数据处理,存储,检索工作中.例如:
- 文件的完整性检查:
- 互联网技术中到处都是用和依靠散列函数,用于网页传输中的各种安全性和正确性检查,包括各种网络认证和检查,各种网络协议
- 计算机安全领域中大量使用散列技术,例如在各种安全协议中.

将散列技术应用于数据的存储和检索,就得到了散列表.实现一个散列表,不但需要选择合适的关键码映射(散列函数),还要考虑由于采用映射方式确定存储和检索位置而带来的各种问题.
#### 散列技术:设计和性质
下标集合通常都远远小于关键码集合的规模.在通常情况下,散列函数h时从一个大集合到一个小集合的映射.在这种情况下,它显然不可能是单射,必然会出现多个不同的关键码被h对应到同一个下标位置的情况.如果实际中出现了这种情况,人们就说这里出现了`冲突`(或者`碰撞`),此时也称$key_1$和$key_2$为h下的`同义词`.根据上面分析,冲突是散列表使用中必然会出现的情况,因此,在考虑散列表的实现时,必须考虑如何解决冲突的问题.

对于一个规模固定的散列表,一般而言,表中的元素越多,出现冲突的可能性也就越大.这里的一个重要概念是`负载因子`.这是一种很有用的度量值,是考察散列表在运行中的性质的一个重要参数,其定义是:$$负载因子\alpha = \frac{散列表中当时的实际数据项数}{散列表的基本存储区能容纳的元素个数}$$如果数据项直接保存在基本存储区里,那么将总会有$\alpha \le 1$.无论如何,负载因子的大小与冲突出现的可能性密切相关:负载因子越大,出现冲突的可能也越大.显然,如果扩大散列表的存储空间(增加可能的存储位置),就可与降低其负载因子,减小出现冲突的概率.但负载因子越小,散列表里空闲空间的比例也就越大.因此这里也有得失权衡的问题.

另一方面,无论负载因子的情况怎样,冲突总是可能出现的.特别是在实现供其他人使用的一般散列表时,开发者无法了解实际使用中的情况,在设计时就必须考虑冲突的处理问题.由于散列表的重要性和广泛应用,人们深入研究散列表的设计,提出了一些冲突处理技术.基于这些处理方式,形成了多种不同的散列表实现结构.

总结一下,如果要基于散列技术实现字典,必须解决两个大问题:散列函数的设计,冲突消解机制.
### 8.2.3 散列函数
在设计字典时,首先应该根据实际需要确定数据项集合,选定相应的关键码集合KEY.为了实现散列表字典,还要确定一个存储位置区间INDEX,例如,选择从0开始的一段下标.KEY和INDEX是两个有限集,分别为将要定义的散列函数的参数域(定义域)和值域,是定义散列函数的基础.从本质上说,任何从KEY到INDEX的全函数f都能满足散列函数的基本要求,因此对任何$key\in KEY,都有f(key)\in INDEX$,函数值落在合法下标范围内.但另一方面,散列函数的选择可能影响出现冲突的概率.

在设计散列函数时,有价值的考虑包括:
- 函数应能把关键码映射到值域INDEX中尽可能大的部分.显然,如果扩大函数值的范围(在INDEX内),出现冲突的可能性就会下降.如果某个下标不是散列函数的可能值,这个位置可能就无法用到.应该尽量避免这种情况.
- 不同关键码的散列值在INDEX里均匀分布,有可能减少冲突.当然,实际情况还域真实数据里不同关键值出现的分布有关.如果不知道关键码的实际分布(例如,开发一个字典库模块时的i情况就是这样),就只能考虑均匀分布.
- 函数的计算比较简单.这一要求很显然:使用散列表的本意是希望提高效率,而计算散列函数的开销是各种操作的开销中的一部分.
#### 用于整数关键码的若干散列方法
**数字分析法**:对于给定的关键码集合,分析所有关键码中各位数字的出现频率,从中选出分布情况较好的若干数字作为散列函数的值.

__折叠法__:将较长的关键码切分为几段,通过某种运算将它们合并.

**中平方法**:先求出关键码的平方,然后取出中间的几位作为散列值.

从上述实例可以看出,对于整数关键码,散列函数的设计有两方面的追求:其一是把较长的关键码映射到较小的区间,另一个就是尽可能消除关键码与映射值之间明显的规律性.通俗地说,`散列函数地映射关系越乱越好,越不清晰越好`.
#### 常用散列函数
- 除余法,适用于整数关键码:关键码key是整数,用key除以某个不大于散列表长度m的整数p,以得到的余数(或者余数加l,由下标开始值确定)作为散列地址.
- 基数转换法,适用于整数或字符串关键码:先考虑整数关键码.取一个正整数r,把关键码看作基数为r的数(r进制的数),将其转换为十进制或二进制数.通常r取素数以减少规律性.